# Model Construction and Evaluation

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Standardization-of-data" data-toc-modified-id="Standardization-of-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Standardization of data</a></span><ul class="toc-item"><li><span><a href="#Training-set" data-toc-modified-id="Training-set-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training set</a></span></li><li><span><a href="#Validation-and-test-set" data-toc-modified-id="Validation-and-test-set-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Validation and test set</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Metrics</a></span></li><li><span><a href="#Random-Forest-Regressor" data-toc-modified-id="Random-Forest-Regressor-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Random Forest Regressor</a></span><ul class="toc-item"><li><span><a href="#Grid-Search-to-find-the-best-RF-hyper-parameters" data-toc-modified-id="Grid-Search-to-find-the-best-RF-hyper-parameters-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Grid Search to find the best RF hyper-parameters</a></span></li><li><span><a href="#Train-RF-to-predict-one-day-at-a-time" data-toc-modified-id="Train-RF-to-predict-one-day-at-a-time-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train RF to predict one day at a time</a></span></li><li><span><a href="#RF-with-feature-selection" data-toc-modified-id="RF-with-feature-selection-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>RF with feature selection</a></span></li></ul></li><li><span><a href="#LSTM" data-toc-modified-id="LSTM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>LSTM</a></span><ul class="toc-item"><li><span><a href="#Building-Model" data-toc-modified-id="Building-Model-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Building Model</a></span></li><li><span><a href="#Training-Model" data-toc-modified-id="Training-Model-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training Model</a></span></li></ul></li><li><span><a href="#GRU" data-toc-modified-id="GRU-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>GRU</a></span></li><li><span><a href="#Stacked-LSTM" data-toc-modified-id="Stacked-LSTM-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Stacked LSTM</a></span></li><li><span><a href="#Submission" data-toc-modified-id="Submission-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Submission</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import feather 
import time
import matplotlib.pyplot as plt
import sys
%matplotlib inline
default_stdout = sys.stdout

# Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM, GRU
from keras.layers.advanced_activations import ELU
from keras import optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error,r2_score,explained_variance_score
from sklearn.feature_selection import mutual_info_regression,SelectKBest
from sklearn.pipeline import Pipeline

Using TensorFlow backend.
/Users/Bato/anaconda/envs/aind-dog/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Bato/anaconda/envs/aind-dog/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [62]:
X_train = feather.read_dataframe("./data/df_x_train.feather")
Y_train = feather.read_dataframe("./data/df_y_train.feather")

In [63]:
x_val = feather.read_dataframe("./data/X_val.feather")
y_val = feather.read_dataframe("./data/y_val.feather")

In [64]:
x_test = feather.read_dataframe("./data/X_test.feather")

In [65]:
weights = np.load("./data/weights.npy")

## Standardization of data

### Training set

In [67]:
train_scaler = StandardScaler()
train_scaler.fit(X_train[:300])
X_train = train_scaler.transform(X_train)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
Y_train = Y_train.as_matrix()

### Validation and test set

In [68]:
val_scaler = StandardScaler()
val_scaler.fit(x_val)
test_scaler = StandardScaler()
test_scaler.fit(x_test)

x_val = val_scaler.transform(x_val)
x_val = x_val.reshape((x_val.shape[0],1,x_val.shape[1]))
x_test = val_scaler.transform(x_test)
x_test = x_test.reshape((x_test.shape[0],1,x_test.shape[1]))

## Metrics

In [69]:
def NWRMSLE(y_true, y_pred, weights, if_list=True):
    """
    y_pred: a list of prediction of length 16. 
    """
    if if_list:
        error = (y_true - np.array(y_pred).squeeze(axis=2).transpose())**2
    else:
        error = (y_true-y_pred)**2
    
    normalized_weighted_error = error.sum(axis=1)*weights
    root_mean = np.sqrt(normalized_weighted_error.sum()/weights.sum()/16)
    
    return root_mean

## Random Forest Regressor

### Grid Search to find the best RF hyper-parameters 

In [ ]:
regr = RandomForestRegressor(n_jobs=-1)
parameters = {'n_estimators':[50,80], 'min_samples_split':[10],
              'max_depth':[10]}
scoring = make_scorer(mean_squared_error)

grid_search_obj = GridSearchCV(estimator=regr, param_grid=parameters, 
                               scoring=scoring, n_jobs=-1,verbose=3)

grid_search_obj.fit(X_train[:,0,:], Y_train[:,:]-Y_train.mean())

best_reg = grid_search_obj.best_estimator_

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] max_depth=10, min_samples_split=10, n_estimators=50 .............
[CV] max_depth=10, min_samples_split=10, n_estimators=50 .............
[CV] max_depth=10, min_samples_split=10, n_estimators=50 .............
[CV] max_depth=10, min_samples_split=10, n_estimators=80 .............


In [14]:
rf_pred = best_reg.predict(x_val[:3000,0,:])+Y_train.mean()
print("Results for validation from Random forest......................................")
print("The NWRMSLE is {0}".format(NWRMSLE(y_val[:3000,:], rf_pred, weights[:3000], if_list=False)))
print("The r2 score is {0}".format(r2_score(y_val[:3000,:], rf_pred, sample_weight=weights[:3000])))
print("The explained variance is {0}".format(explained_variance_score(y_val[:3000],rf_pred,sample_weight=weights[:3000])))
print("\n")

Results for validation from Random forest......................................
The NWRMSLE is 0.6267107608443206
The r2 score is 0.5355330073373146
The explained variance is 0.5589823430786114




### Train RF to predict one day at a time

In [52]:
def run_RF_one_day(best_reg, X, Y, w, xval, yval, xtest):
    pred_val = []
    pred_test = []
    
    parameters = best_reg.get_params()
    n = parameters['n_estimators']
    min_samples_split = parameters['min_samples_split']
    max_depth = parameters['max_depth']
    
    for i in range(0, 3):
        print("Predicting %d day "%i)
        print("="*50)
        
        y = Y[:,i] - Y[:,i].mean()
        reg = RandomForestRegressor(n_estimators=n, min_samples_split=min_samples_split,
                                   max_depth=max_depth,n_jobs=-1, verbose=1)
        reg.fit(X, y, sample_weight=w)
        
        pred_val.append(reg.predict(xval)+Y[:,i].mean())
        pred_test.append(reg.predict(xtest)+Y[:,i].mean())
        
    return pred_val, pred_test

In [97]:
print("Training RF model to one day.............................................")
start_time = time.time()
pred_val, test_val = run_RF_one_day(best_reg, X_train[:3000,0,:], Y_train[:3000,:], weights[:3000],
                                    x_val[:3000,0,:], y_val[:3000], x_test[:3000,0,:])
print("The time we used to train RF is {0} minutes".format((time.time()-start_time)/60))
print("The NWRMSLE of validation set is {0}".format(NWRMSLE(y_val[:3000,:3], np.array(pred_val).T, weights[:3000], if_list=False)))
print("The r2 score of validation set with RF is {0}".format(r2_score(y_val[:3000,:3], np.array(pred_val).T, sample_weight=weights[:3000])))
print("The explained variance is {0}".format(explained_variance_score(y_val[:3000,:3],np.array(pred_val).T,sample_weight=weights[:3000])))
print("\n")


Training RF model to one day.............................................
Predicting 0 day 


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.7s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


Predicting 1 day 


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


Predicting 2 day 


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.8s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


The time we used to train RF is 0.3534686009089152 minutes
The NWRMSLE of validation set is 0.6209006065180679
The r2 score of validation set with RF is 0.5812058871282513
The explained variance is 0.5873526703771615




[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


### RF with feature selection

In [98]:
def RF_feature_selection(best_reg, X, Y, w, xval, yval, xtest):
    pred_val = []
    pred_test = []
    
    parameters = best_reg.get_params()
    n = parameters['n_estimators']
    min_samples_split = parameters['min_samples_split']
    max_depth = parameters['max_depth']
    
    for i in range(0, 3):
        print("Predicting %d day "%i)
        print("="*50)
        
        y = Y[:,i] - Y[:,i].mean()
        reg = RandomForestRegressor(n_estimators=n, min_samples_split=min_samples_split,
                                   max_depth=max_depth,n_jobs=-1, verbose=1)
        
        reg_filter = SelectKBest(mutual_info_regression, k=300)
        
        new_reg = Pipeline([('filter', reg_filter),('reg', reg)])
        new_reg.fit(X, y, sample_weight=w)
        
        pred_val.append(reg.predict(xval)+Y[:,i].mean())
        pred_test.append(reg.predict(xtest)+Y[:,i].mean())
        
    return pred_val, pred_test

In [99]:
print("Training RF model with feature selection.............................................")
start_time = time.time()
pred_val, test_val = run_RF_one_day(best_reg, X_train[:3000,0,:], Y_train[:3000,:], weights[:3000],
                                    x_val[:3000,0,:], y_val[:3000], x_test[:3000,0,:])
print("The time we used to train RF is {0} minutes".format((time.time()-start_time)/60))
print("The NWRMSLE of validation set is {0}".format(NWRMSLE(y_val[:3000,:3], np.array(pred_val).T, weights[:3000], if_list=False)))
print("The r2 score of validation set with RF is {0}".format(r2_score(y_val[:3000,:3], np.array(pred_val).T, sample_weight=weights[:3000])))
print("The explained variance is {0}".format(explained_variance_score(y_val[:3000,:3],np.array(pred_val).T,sample_weight=weights[:3000])))
print("\n")


Training RF model with feature selection.............................................
Predicting 0 day 


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


Predicting 1 day 


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    8.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


Predicting 2 day 


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


The time we used to train RF is 0.3785074989000956 minutes
The NWRMSLE of validation set is 0.6225411357043847
The r2 score of validation set with RF is 0.5792547560972058
The explained variance is 0.586899520077849




## LSTM

### Building Model

In [ ]:
def build_LSTM(X):
    
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(128))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(64))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(32))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(16))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(1))
    
    return model

### Training Model

In [ ]:
def run_LSTM(X, Y, xval, yval, xtest, weights, epoches):
    pred_val = []
    pred_test = []
    for i in range(0, 16):
        print("Predicting %d day "%i)
        print("="*50)
        
        y = Y[:,i] - Y[:,i].mean()
        model = build_LSTM(X)
        opt = optimizers.Adam(lr=0.001)
        model.compile(loss='mse', optimizer=opt, metrics=['mse'])
        
        call_backs = [
            EarlyStopping(monitor='val_loss', patience=10, verbose=0),
            ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, 
                              verbose=1, mode='min')
        ]
        
        model.fit(X, y, batch_size=1024, epochs=epoches, verbose=2,
                 sample_weight=weights, validation_data=(xval, yval[:,i]-Y[:,i].mean()),
                 callbacks=call_backs)
        
        pred_val.append(model.predict(xval)+Y[:,i].mean())
        pred_test.append(model.predict(xtest)+Y[:,i].mean())
        
    return model, pred_val, pred_test
    

In [ ]:
print("Training LSTM model.............................................")
start_time = time.time()
model, pred_val, test_val = run_LSTM(X_train, Y_train, x_val, y_val, 
                                     x_test, weights, 1000)
print("The time we used to train LSTM network is {0} minutes".format((time.time()-start_time)/60))



In [40]:
print("The NWRMSLE of validation set is {0}".format(NWRMSLE(y_val, lstm_pred_val, weights[:170810])))
print("The r2 score of validation set with LSTM is {0}".format(r2_score(y_val, np.array(lstm_pred_val)[:,:,0].T, sample_weight=weights[:170810])))
print("The explained variance is {0}".format(explained_variance_score(y_val,np.array(lstm_pred_val)[:,:,0].T,sample_weight=weights[:170810])))
print("\n")

The NWRMSLE of validation set is 0.5886397111899362
The r2 score of validation set with LSTM is 0.6966021563558484
The explained variance is 0.6998464651219788




In [116]:
# sys.stdout = default_stdout

## GRU

In [ ]:
def build_GRU(X):
    
    model = Sequential()
    model.add(GRU(256, input_shape=(X.shape[1],X.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(128))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(64))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(32))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(16))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(1))
    
    return model

In [ ]:
def run_GRU(X, Y, xval, yval, xtest, weights, epoches):
    pred_val = []
    pred_test = []
    for i in range(0, 16):
        print("Predicting %d day "%i)
        print("="*50)
        
        y = Y[:,i] - Y[:,i].mean()
        model = build_GRU(X)
        opt = optimizers.Adam(lr=0.001)
        model.compile(loss='mse', optimizer=opt, metrics=['mse'])
        
        call_backs = [
            EarlyStopping(monitor='val_loss', patience=10, verbose=0),
            ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, 
                              verbose=1, mode='min')
        ]
        
        model.fit(X, y, batch_size=1024, epochs=epoches, verbose=2,
                 sample_weight=weights, validation_data=(xval, yval[:,i]-Y[:,i].mean()),
                 callbacks=call_backs)
        
        pred_val.append(model.predict(xval)+Y[:,i].mean())
        pred_test.append(model.predict(xtest)+Y[:,i].mean())
        
    return model, pred_val, pred_test
    

In [ ]:
print("Training GRU model.............................................")
start_time = time.time()
model, pred_val, test_val = run_GRU(X_train, Y_train, x_val, y_val, 
                                     x_test, weights, 1000)
print("The time we used to train GRU network is {0} minutes".format((time.time()-start_time)/60))
print("The NWRMSLE of validation set is {0}".format(NWRMSLE(y_val, pred_val, weights)))
print("The r2 score of validation set with GRU is {0}".format(r2_score(y_val, pred_val, sample_weight=weights)))
print("The explained variance is {0}".format(explained_variance_score(y_val,pred_val,sample_weight=weights)))
print("\n")

## Stacked LSTM

In [ ]:
def build_stackedLSTM(X):
    
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]),
             return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(LSTM(128))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(64))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(32))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(16))
    model.add(ELU())
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(Dense(1))
    
    return model

In [ ]:
def run_stackedLSTM(X, Y, xval, yval, xtest, weights, epoches):
    pred_val = []
    pred_test = []
    for i in range(0, 16):
        print("Predicting %d day "%i)
        print("="*50)
        
        y = Y[:,i] - Y[:,i].mean()
        model = build_stackedLSTM(X)
        opt = optimizers.Adam(lr=0.001)
        model.compile(loss='mse', optimizer=opt, metrics=['mse'])
        
        call_backs = [
            EarlyStopping(monitor='val_loss', patience=10, verbose=0),
            ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, 
                              verbose=1, mode='min')
        ]
        
        model.fit(X, y, batch_size=1024, epochs=epoches, verbose=2,
                 sample_weight=weights, validation_data=(xval, yval[:,i]-Y[:,i].mean()),
                 callbacks=call_backs)
        
        pred_val.append(model.predict(xval)+Y[:,i].mean())
        pred_test.append(model.predict(xtest)+Y[:,i].mean())
        
    return model, pred_val, pred_test
    

In [ ]:
print("Training stackedLSTM model.............................................")
start_time = time.time()
model, pred_val, test_val = run_stackedLSTM(X_train, Y_train, x_val, y_val, 
                                     x_test, weights, 1000)
print("The time we used to train stackedLSTM network is {0} minutes".format((time.time()-start_time)/60))
print("The NWRMSLE of validation set is {0}".format(NWRMSLE(y_val, pred_val, weights)))
print("The r2 score of validation set with stacked LSTM is {0}".format(r2_score(y_val, pred_val, sample_weight=weights)))
print("The explained variance is {0}".format(explained_variance_score(y_val,pred_val,sample_weight=weights)))
print("\n")

## Submission

In [55]:
import pdb
def create_submission(df_2017, df_test):
    y_test = np.load("./res/lstm_pred_test_1.npy")
    y_test = y_test.squeeze(axis=2).transpose()
    df_preds = pd.DataFrame(y_test, index=df_2017.index,
                           columns=pd.date_range("2017-08-16", periods=16)).stack().to_frame("unit_sales")

    df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
    
    submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
    submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
    submission.to_csv('./submission/lstm_pred_test_1.csv', float_format='%.4f', index=None)
    
    return 

In [51]:
train_raw = feather.read_dataframe("./data/train_raw.feather")
train_raw['unit_sales'] = train_raw['unit_sales'].apply(func=lambda x:np.log1p(x) if float(x)>0 else 0)
df_2017 = train_raw.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

df_test = pd.read_csv(
    "./data/test.csv",
    converters={"onpromotion": lambda p:int(p==True)},
    parse_dates=["date"]
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [54]:
create_submission(df_2017, df_test)